# %% [markdown]
# 🧰 Install deps (first run only)

# %% [markdown]
# 🔐 Auth & Config

In [5]:
# %%
from pathlib import Path
import json
import pandas as pd
import gspread
from google.oauth2.service_account import Credentials
from typing import Dict, Any, List, Tuple

# ========= CONFIG =========
SERVICE_ACCOUNT_FILE = "key.json"   # <-- change
SPREADSHEET_ID       = "1I4FHncl40_xx1Udc_Q2rWWWvpL6xaMlpJyY90WBftag"            # <-- change
WORKSHEET_NAME       = "Monsters"                        # <-- change
JSON_DIR             = Path("data/Monsters")                 # folder with *.json

MATCH_ON       = ["Name"]   # use ["Name"] if you don't track Source
ADD_IF_MISSING = True                 # append new rows if not found
DRY_RUN        = False                # True = preview only; no sheet writes
# =========================

SCOPES = ["https://www.googleapis.com/auth/spreadsheets"]
creds = Credentials.from_service_account_file(SERVICE_ACCOUNT_FILE, scopes=SCOPES)
gc = gspread.authorize(creds)

ws = gc.open_by_key(SPREADSHEET_ID).worksheet(WORKSHEET_NAME)

# %% [markdown]
# 📥 Load worksheet as DataFrame & build indices

In [6]:
# %%
values = ws.get_all_values()  # row 1 = header
if not values:
  raise RuntimeError("Worksheet is empty—please add a header row first.")

header = values[0]
rows   = values[1:]
df     = pd.DataFrame(rows, columns=header)

def norm(v: str) -> str:
  return (v or "").strip()

def key_of_row(row: pd.Series) -> Tuple[str, ...]:
  return tuple(norm(row.get(k, "")) for k in MATCH_ON)

row_index = { key_of_row(r): i for i, r in df.iterrows() }  # 0-based DataFrame index

# quick helpers
def ensure_columns_exist(cols: List[str]) -> None:
  """Add any missing columns to the header row in the Sheet and our local df."""
  global header, df
  missing = [c for c in cols if c not in header]
  if not missing:
    return
  new_header = header + missing
  if not DRY_RUN:
    ws.update('1:1', [new_header])  # replace header row
  # update local structures
  for c in missing:
    df[c] = ""
  header = new_header

# %% [markdown]
# 🧩 JSON schema helpers (matches your Title-Case fields)

In [7]:
# %%
from typing import Optional

SPEED_COLS = [
  "Speed (Walking)",
  "Speed (Swimming)",
  "Speed (Flying)",
  "Speed (Burrowing)",
  "Speed (Climbing)",
]

def fmt_hp(hp: Any) -> Tuple[str, str]:
  if isinstance(hp, dict):
    return (str(hp.get("Value", "")), str(hp.get("Notes", "")))
  return ("", "")

def fmt_ac(ac: Any) -> Tuple[str, str]:
  if isinstance(ac, dict):
    return (str(ac.get("Value","")), str(ac.get("Notes","")))
  if isinstance(ac, (int, float, str)):
    return (str(ac), "")
  return ("", "")

def fmt_speed_list(blip: Any) -> Dict[str, str]:
  lst = blip[0].split(", ")
  out = {c: "" for c in SPEED_COLS}
  if not isinstance(lst, list): return out
  for token in map(str, lst):
    low = token.lower()
    if   "fly"   in low: out["Speed (Flying)"]    = token.replace("fly ", "")
    elif "swim"  in low: out["Speed (Swimming)"]  = token.replace("swim ", "")
    elif "burrow"in low: out["Speed (Burrowing)"] = token.replace("burrow ", "")
    elif "climb" in low: out["Speed (Climbing)"]  = token.replace("climb ", "")
    else:
      out["Speed (Walking)"] = token if not out["Speed (Walking)"] else f"{out['Speed (Walking)']}, {token}"
  return out

def fmt_abilities(d: Any) -> Dict[str, str]:
  out: Dict[str, str] = {}
  if isinstance(d, dict):
    for j,c in [("Str","STR"),("Dex","DEX"),("Con","CON"),("Int","INT"),("Wis","WIS"),("Cha","CHA")]:
      if j in d:
        score = d[j]
        out[c] = score
        mod_col = f"{c} Mod"
        if mod_col in header:
          try:
            mod = (int(score)-10)//2
            out[mod_col] = f"{mod:+d}"
          except Exception:
            pass
  return out

def fmt_bonus_list(lst: Any) -> str:
  if not isinstance(lst, list): return ""
  parts = []
  for it in lst:
    if isinstance(it, dict):
      n = str(it.get("Name","")).strip()
      m = it.get("Modifier","")
      if n:
        parts.append(f"{n} {m:+d}" if isinstance(m,int) else f"{n} {m}")
  return ", ".join(parts)

def fmt_list(v: Any) -> str:
  return ", ".join(map(str, v)) if isinstance(v, list) else (str(v) if v else "")

def fmt_blocks(blocks: Any) -> str:
  if not isinstance(blocks, list): return ""
  lines = []
  for b in blocks:
    if isinstance(b, dict):
      nm = str(b.get("Name","")).strip()
      ct = str(b.get("Content","")).strip()
      if nm and ct: lines.append(f"{nm}. {ct}")
      elif ct:      lines.append(ct)
      elif nm:      lines.append(nm)
  return "\n".join(lines)

def monster_to_row_updates(path: Path, m: Dict[str, Any]) -> Dict[str, Any]:
  out: Dict[str, Any] = {}

  # Identity
  name = (m.get("Name") or m.get("name") or path.stem).strip()
  out["Name"]   = name
  out["Source"] = str(m.get("Source","")).strip()

  # Basics
  type_text = str(m.get("Type","") or m.get("type","")).strip()
  parts = type_text.split(" ")
  size, creature_type, alignment = (parts + ["", "", ""])[:3]
  for col in ["Classic 5e Type"]:
    if col in header and type_text:
      out[col] = type_text
      break
  if "Type" in header and m.get("Type"): out["Type"] = creature_type
  if "Size" in header: out["Size"] = size
  if "Alignment" in header: out["Alignment"] = alignment

  # AC / HP
  ac_val, ac_notes = fmt_ac(m.get("AC"))
  if "Armor Class" in header and ac_val: out["Armor Class"] = ac_val
  if "Armor Type"  in header and ac_notes: out["Armor Type"] = ac_notes

  hp_val, hd_notes = fmt_hp(m.get("HP"))
  hit_dice_parts = hd_notes.split(" + ")
  if "Hit Points" in header and hp_val: out["Hit Points"] = hp_val
  if "Hit Dice"   in header and hd_notes: out["Hit Dice"] = hd_notes[0]

  # Speed
  out.update({k:v for k,v in fmt_speed_list(m.get("Speed")).items() if k in header and v})

  # Abilities
  out.update({k:v for k,v in fmt_abilities(m.get("Abilities")).items() if k in header})

  # Saves / Skills
  saves = fmt_bonus_list(m.get("Saves"))
  if "Saving Throws" in header and saves: out["Saving Throws"] = saves
  elif "Saves" in header and saves:       out["Saves"] = saves

  skills = fmt_bonus_list(m.get("Skills"))
  if "Skills" in header and skills: out["Skills"] = skills

  # Damage & Conditions
  for jkey, col in [
    ("DamageVulnerabilities", "Damage Vulnerabilities"),
    ("DamageResistances",     "Damage Resistances"),
    ("DamageImmunities",      "Damage Immunities"),
    ("ConditionImmunities",   "Condition Immunities"),
  ]:
    txt = fmt_list(m.get(jkey))
    if txt and col in header:
      out[col] = txt

  # Senses / Languages / Passive
  if "Passive Perception" in header and "Passive" in m:
    out["Passive Perception"] = str(m["Passive"])

  # CR / PB
  cr = str(m.get("Challenge","")).strip()
  if cr and "CR (Challenge Rating)" in header:
    out["CR (Challenge Rating)"] = cr

  pb = str(m.get("ProficiencyBonus", m.get("Proficiency",""))).strip()
  if pb and "Proficiency Bonus" in header:
    out["Proficiency Bonus"] = pb

  # Text blocks
  blocks = [
    ("Traits","Traits"), ("Actions","Actions"), ("Reactions","Reactions"),
    ("BonusActions","Bonus Actions"), ("LegendaryAction","Legendary Action"),
    ("LairAction","Lair Action"), ("MythicActions","Mythic Actions"),
  ]
  for jkey, col in blocks:
    txt = fmt_blocks(m.get(jkey))
    if txt and col in header:
      out[col] = txt

  return out

# %% [markdown]
# 📂 Read JSON files, compute updates, and write to Google Sheet

In [8]:
from gspread.utils import rowcol_to_a1
# %%
def json_files_in_folder(folder: Path) -> List[Path]:
  return [p for p in folder.glob("*.json") if p.is_file()]

# Ensure we know about all possible columns we might write
ensure_columns_exist([
  "Name","Source","Size","Classic 5e Type","Type","New Type","Alignment",
  "Armor Class","Armor Type","Hit Points","Hit Dice",
  *SPEED_COLS,
  "STR","DEX","CON","INT","WIS","CHA","STR Mod","DEX Mod","CON Mod","INT Mod","WIS Mod","CHA Mod",
  "Saving Throws","Skills",
  "Damage Vulnerabilities","Damage Resistances","Damage Immunities","Condition Immunities",
  "Senses","Languages","Passive Perception",
  "CR (Challenge Rating)","Proficiency Bonus",
  "Traits","Actions","Reactions","Bonus Actions","Legendary Action","Lair Action","Mythic Actions",
  "Description",
])

# Prepare updates
to_update = 0
to_append = 0
row_updates_payload = []  # list of {"range": "A5:AZ5", "values": [[...]]}
new_rows = []             # list of full row arrays for append_rows

# Quick col index map
col_index = {name: i for i, name in enumerate(header)}  # 0-based

def row_as_full_array(existing: pd.Series, patch: Dict[str, Any]) -> List[str]:
  # Merge existing row + patch into a full list aligned to header
  vals = [existing.get(h, "") for h in header]
  for col, v in patch.items():
    if col in col_index:
      vals[col_index[col]] = str(v)
  return [str(v) for v in vals]

def empty_row_from_patch(patch: Dict[str, Any]) -> List[str]:
  vals = ["" for _ in header]
  for col, v in patch.items():
    if col in col_index:
      vals[col_index[col]] = str(v)
  return vals

# Iterate files
for path in json_files_in_folder(JSON_DIR):
  try:
    m = json.loads(path.read_text(encoding="utf-8"))
  except Exception as e:
    print(f"Skipped {path.name}: {e}")
    continue

  patch = monster_to_row_updates(path, m)
  key = tuple(norm(patch.get(k, "")) for k in MATCH_ON)

  if key in row_index:
    # existing row -> compute merged row and send exact row update (A1-based range)
    df_idx = row_index[key]
    merged = row_as_full_array(df.iloc[df_idx], patch)
    # Sheet row number (header is row 1)
    sheet_row = df_idx + 2
    start_cell = rowcol_to_a1(sheet_row, 1)
    end_cell   = rowcol_to_a1(sheet_row, len(header))
    rng = f"{start_cell}:{end_cell}"
    row_updates_payload.append({"range": rng, "values": [merged]})
    to_update += 1
    for k, v in patch.items():
      df.at[df_idx, k] = v
  else:
    if ADD_IF_MISSING:
      new_rows.append(empty_row_from_patch(patch))
      to_append += 1
    else:
      print(f"Not found (skipped): {key}")

print(f"Planned: update {to_update} row(s), append {to_append} row(s).")

if DRY_RUN:
  print("DRY_RUN=True — not writing to Google Sheets.")
else:
  # Batch row updates
  if row_updates_payload:
    ws.batch_update(row_updates_payload, value_input_option="USER_ENTERED")
    print(f"✅ Updated {len(row_updates_payload)} existing row(s).")
  # Append new
  if new_rows:
    ws.append_rows(new_rows, value_input_option="USER_ENTERED")
    print(f"✅ Appended {len(new_rows)} new row(s).")
  print("Done.")

Skipped Hippantrope.json: Expecting value: line 1 column 1 (char 0)
Planned: update 112 row(s), append 1 row(s).
✅ Updated 112 existing row(s).
✅ Appended 1 new row(s).
Done.
